# digital signatures

In [ ]:
# Install a pip package in the current Jupyter kernel
# from: https://note.nkmk.me/en/python-pip-install-requirements/
import sys
!{sys.executable} -m pip install pycoin==0.80

In [10]:
# Adapted from: S. Nakov, "Practical Cryptography for Developers"
# https://cryptobook.nakov.com/digital-signatures/ecdsa-sign-verify-examples

# Uses ECDSA (using the elliptic curve secp256k1 + SHA3-256 hash algorithm)

from pycoin.ecdsa import generator_secp256k1, sign as signECDSA, verify as verifyECDSA
import hashlib, secrets

def sha3_256Hash(msg):
    hashBytes = hashlib.sha3_256(msg.encode("utf8")).digest()
    return int.from_bytes(hashBytes, byteorder="big")

def genKeyPair():
    privKey = secrets.randbelow(generator_secp256k1.order())
    pubKey = (generator_secp256k1 * privKey).pair()
    return [privKey, pubKey]

def sign(msg, privKey):
    msgHash = sha3_256Hash(msg)
    signature = signECDSA(generator_secp256k1, privKey, msgHash)
    return signature

def verify(msg, signature, pubKey):
    msgHash = sha3_256Hash(msg)
    valid = verifyECDSA(generator_secp256k1, pubKey, msgHash, signature)
    return valid

## Key generation

In [11]:
[privKey, pubKey] = genKeyPair()

{"privKey": hex(privKey),
 "pubKey": {
    "x": hex(pubKey[0]),
    "y": hex(pubKey[1])
 }
}


{'privKey': '0x691e0ff11a797f02bf4654d95e86c69171fdeef7c9638b79f5e5df85d2932c19',
 'pubKey': {'x': '0x1a16b7579cf04615c501d2a65e2e98f0c4b498de395d26714f8be99595b6b977',
  'y': '0x8d055c006c24102e22b26828d93be85a161865a33b6983a250f3f1cad0a0222f'}}

## Signing a message

Sign a message using the private key:

In [12]:
msg = "alice pays bob $10"

signature = sign(msg, privKey)

{"message": msg,
 "signature": {
    "r": hex(signature[0]),
    "s": hex(signature[1])
}}

{'message': 'alice pays bob $10',
 'signature': {'r': '0xc8dacf19c758e4a2556d9ffbab95254e6a7782df00d61c4f3d4c3e4af6663b5d',
  's': '0x4475a3c4a8e1e6cf647a252dcd4b90eb3ac896a2cd8800a1d7ff925a18ef05e8'}}

## Verifying a signature

Verify signature of a valid signed message:

In [13]:
valid = verify(msg, signature, pubKey)

{"message": msg,
 "signature valid?": valid}

{'message': 'alice pays bob $10', 'signature valid?': True}

Verify signature of a tampered message:


In [14]:
msg2 = "alice pays bob $100"

valid = verify(msg2, signature, pubKey)

{"message": msg2,
 "signature valid?": valid}

{'message': 'alice pays bob $100', 'signature valid?': False}